In [22]:
"""
===================================================================================
Importing and Initializing
===================================================================================
"""
import easygopigo3
import gopigo3
import easysensors
egpg = easygopigo3.EasyGoPiGo3()
gpg = gopigo3.GoPiGo3()
egpg.reset_all()

# IMPORT TOOLS FOR MAKING JUPYTER EASIER TO READ
from IPython.display import clear_output
from EDL_Jupyter_resources import HiddenPrints

# IMPORT USFUL TOOLS
import time
import math
import numpy as np
import matplotlib.pyplot as plt
hiddenprints=HiddenPrints()


# INITIALIZE A BUTTON (TELL THE ROBOT THERE IS A BUTTON)
my_button = egpg.init_button_sensor("AD2")
PRESSED = 1


# SETUP A DICTIONARY FOR MORSE CODE TRANSLATION
MORSE_CODE = {     '.-':'A',  '-...':'B', 
                 '-.-.':'C',   '-..':'D',        '.':'E', 
                 '..-.':'F',   '--.':'G',     '....':'H', 
                   '..':'I',  '.---':'J',      '-.-':'K', 
                 '.-..':'L',    '--':'M',       '-.':'N', 
                  '---':'O',  '.--.':'P',     '--.-':'Q', 
                  '.-.':'R',   '...':'S',        '-':'T', 
                  '..-':'U',  '...-':'V',      '.--':'W', 
                 '-..-':'X',  '-.--':'Y',     '--..':'Z', 
                '.----':'1', '..---':'2',    '...--':'3', 
                '....-':'4', '.....':'5',    '-....':'6', 
                '--...':'7', '---..':'8',    '----.':'9', 
                '-----':'0', '--..--':',',  '.-.-.-':'.', 
               '..--..':'?',  '-..-.':'/',  '-....-':'-', 
                '-.--.':'(', '-.--.-':')',       ' ':' ',
                'END':""} 

"""
===================================================================================
SOME CUSTOM FUNCTIONS
===================================================================================
"""

def record_button_press_time():
    # This function measures how long a button is pressed for.
    while not my_button.read() == PRESSED:
        # Wait for press
        time.sleep(.01)
    down_time = time.time()
    while my_button.read() == PRESSED:
        # Wait for release
        time.sleep(.01)
    up_time = time.time()     
    return up_time - down_time


def collect_training_presses(number, type_name):
    # This function collects "number" samples of "type_name" and records the durations in an array "times"
    times = np.zeros(number)
    for press_num in range(number):
        print("Waiting for ", type_name, " PRESS ", press_num+1, " of ", number)
        times[press_num] = record_button_press_time()
    return times

def classify(time, dot_average, dash_average):
    # This function should:
    # RETURN "-" if the time is for a dash, or "." if the time is for a dot.
    """
    TO DO:
    In order to decide if the "time" given, is a dot or a dash, you need to calculate the number 
    is more similiar to the dot average or the dash average. There are multiple ways to do this.
    """
    dot_distance = abs(time-dot_average)
    dash_distance = abs(time-dash_average)
    if dot_distance <= dash_distance:
        return "."
    else:
        return "-"

    
    
"""
===================================================================================
CODE THAT RUNS
===================================================================================
""" 

"""
Collect LABELED TRAINING samples
"""
train = 1 # This is a 1 if the system needs to be trained
if not dot_average is None and not dash_average is None:
    # If it is already trained, allow the user to decide whether to train again.
    train = input("Do you want to retrain? 1 = Yes, 0 = No ")
    try:
        train = int(train)
    except:
        print("*** Please provide an integer ***")
        
if train == 1:
    # If training or retraining:
    # Ask how many samples to collect.
    collect_number = input("How many training samples do you want to give? ")
    try:
        collect_number = int(collect_number)
    except:
        print("*** Please provide an integer ***")
    
    #Collect the samples
    print("DOTS")
    dot_times = collect_training_presses(collect_number, "Dot")

    print("DASHES")
    dash_times = collect_training_presses(collect_number, "Dash")

    print("Dot Times: ", dot_times)
    print("Dash Times: ", dash_times)


"""
TO DO: Calculate the Average Times for Dashes and Dots
 np.mean(ARRAY) will return the mean of an array
 np.sum(ARRAY) will return the sum of an array
 np.size(ARRAY) will return the number of items in an array
"""
dot_average = np.mean(dot_times)
dash_average = np.mean(dash_times)

print("Dot Average: ", dot_average)
print("Dash Average: ", dash_average)

print("===================")
print("STARTING IN")
time.sleep(1)
print("3...")
time.sleep(1)
print("2...")
time.sleep(1)
print("1...")
time.sleep(1)
print("GO")
print("===================")

last_press_time = time.time()
space_time = False
message = [] # Initialize a blank message
letter = "" # Initialize a blank message


"""
ONCE TRAINED THE FOLLOWING LOOP WILL COLLECT A MESSAGE
"""
while True: 
    
    # If there is a long pause, calculate the message.
    if not my_button.read() == PRESSED and time.time() - last_press_time > 8*dash_average:
        if message[-1] == " ":
            print("END MESSAGE")
            print("===================")

            message[-1] = "END"          
            break
    
    # If there is a medium pause, add a space.
    elif not my_button.read() == PRESSED and time.time() - last_press_time > 4*dash_average:
        if letter == "" and not message[-1] == " " :
            print("SPACE")
            print("NEXT LETTER ===========")
            message.append(" ")
          
        
    # If there is a short pause, go to next letter.
    elif not my_button.read() == PRESSED and time.time() - last_press_time > 2*dash_average:
        if not letter == "":
            print("NEXT LETTER ===========")
            message.append(letter)
            letter = "" 
            
    # If the button is pressed, decide if it is a dot or a dash.
    elif my_button.read() == PRESSED:
        down_time = time.time()
        while my_button.read() == PRESSED:
            time.sleep(.01)
        up_time = time.time()
        press_time = up_time - down_time
        print("Saw: ", classify(press_time, dot_average, dash_average))
        letter = letter + classify(press_time, dot_average, dash_average)
        last_press_time = up_time
    
print(message)

"""Translate the Morse Code into Letters"""
message_as_letters = ""
for code in message:
    try:
        message_as_letters = message_as_letters + MORSE_CODE[code]
    except:
        print("DID NOT RECOGNIZE: ", code)
print("===================")
print("Message: ", message_as_letters)

Do you want to retrain? 1 = Yes, 0 = No 1
How many training samples do you want to give? 3
DOTS
Waiting for  Dot  PRESS  1  of  3
Waiting for  Dot  PRESS  2  of  3
Waiting for  Dot  PRESS  3  of  3
DASHES
Waiting for  Dash  PRESS  1  of  3
Waiting for  Dash  PRESS  2  of  3
Waiting for  Dash  PRESS  3  of  3
Dot Times:  [0.08324599 0.07263875 0.06303072]
Dash Times:  [0.91943121 0.76674294 0.81841016]
Dot Average:  0.07297182083129883
Dash Average:  0.8348614374796549
STARTING IN
3...
2...
1...
GO
Saw:  .
Saw:  .
Saw:  .
NEXT LETTER ===========
Saw:  -
Saw:  -
Saw:  -
NEXT LETTER ===========
Saw:  .
Saw:  .
Saw:  .
NEXT LETTER ===========
SPACE
NEXT LETTER ===========
END MESSAGE
['...', '---', '...', 'END']
Message:  SOS
